In [1]:
import os
import itk
import sys
import time
import ants
import json
# from itkwidgets import view --> This seems to cause the issue when an error happened, the kernel disconnects and stays in a zombie state
# import SimpleITK as sitk  # An alternative to itk
import glob
import shutil
import numpy as np
# import pydicom as pyd
# import dicom2nifti
from directory_tree import display_tree  # Nice tool to display directory trees (https://pypi.org/project/directory-tree/)

from datetime import timedelta

In [2]:
def list_folder_content(path, show_hidden=False):
    if show_hidden:
        ddfldrlst = os.listdir(path)
    else:
        ddfldrlst = list(filter(lambda item: not item.startswith('.'),os.listdir(path)))      
    return ddfldrlst

def display_folder_list(file_list):
    print('\n'.join(f'[{idx}] - {file_idx}' for idx, file_idx in enumerate(file_list)))

def get_path_to_process(full_path):
    print('Folder content:')
    print(display_tree(full_path, header=True, string_rep=True, show_hidden=False, max_depth=2))
    folder_content = list_folder_content(full_path)
    # Ideally we'll have only one sub-folder inside the PreTreatment folder. If more than one, then we have to choose, but by default, we'll select the first one.
    idx_reg = 0
    if len(folder_content) > 1:
        display_folder_list(folder_content)
        idx_sel = input(f'Select the folder with the dataset_to_process to process (0-{len(folder_content)-1} or just press Enter to proceed with sub-folder {folder_content[idx_reg]}):')
        if idx_sel:
            idx_reg = int(idx_sel)
    path2data = os.path.join(full_path, folder_content[idx_reg])
    print(f'Will process {folder_content[idx_reg]}')
    return path2data

def check_time_points(path_to_check, nmax = 6, verbose=False):
    if path_to_check is not None:
        nr_of_folders = list_folder_content(path_to_check)
        print(f'Folder {path_to_check} seems Ok' if len(nr_of_folders)== nmax else f'Error! Check path {path_to_check} is the correct one')
        if verbose:
            print('Listing folder content:')
            display_tree(path_to_check, max_depth=1)
        return nr_of_folders if len(nr_of_folders) == nmax else None
    else:
        return None

def add_prefix_to_filename(full_path, prefix=None):
    # Assume the last part of the path is the filename (with extension)
    file_path, file_name_ext = os.path.split(full_path)
    if prefix:
        updated_filename = '_'.join([prefix, file_name_ext])
        return os.path.join(file_path, updated_filename)
    else:
        return prefix
    
def itk2ants(itkInput):
    antsOutput = ants.from_numpy(itk.GetArrayFromImage(itkInput).T, 
                                 origin=tuple(itkInput.GetOrigin()), 
                                 spacing=tuple(itkInput.GetSpacing()), 
                                 direction=np.array(itkInput.GetDirection()))
    
    return antsOutput

def ants2itk(antsInput):
    itkOutput = itk.GetImageFromArray(antsInput.numpy().T)
    itkOutput.SetOrigin(antsInput.origin)
    itkOutput.SetSpacing(antsInput.spacing)
    itkOutput.SetDirection(antsInput.direction)

    return itkOutput

def getenv():
    """
    Requires sys and os modules:
    import sys
    import os
    Possible values for sys.platform are (https://docs.python.org/3/library/sys.html & https://stackoverflow.com/questions/446209/possible-values-from-sys-platform)
    ┍━━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━┑
    │  System             │ Value               │
    ┝━━━━━━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━━━━━━┥
    │ Linux               │ linux or linux2 (*) │
    │ Windows             │ win32               │
    │ Windows/Cygwin      │ cygwin              │
    │ Windows/MSYS2       │ msys                │
    │ Mac OS X            │ darwin              │
    │ OS/2                │ os2                 │
    │ OS/2 EMX            │ os2emx              │
    │ RiscOS              │ riscos              │
    │ AtheOS              │ atheos              │
    │ FreeBSD 7           │ freebsd7            │
    │ FreeBSD 8           │ freebsd8            │
    │ FreeBSD N           │ freebsdN            │
    │ OpenBSD 6           │ openbsd6            │
    │ AIX                 │ aix (**)            │
    ┕━━━━━━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━┙
    """
    if sys.platform == 'win32':
        env_home = 'HOMEPATH'
    elif (sys.platform == 'darwin') | (sys.platform == 'linux'):
        env_home = 'HOME'
    HOMEPATH = os.getenv(env_home)
    
    return HOMEPATH

def check_path_exist(path, file=False):
    """
    Flag FILE indicates the path contains a file name (FLAG=TRUE) or the path only points to a folder (FLAG=FALSE (Default))
    """
    if file:
        is_path = os.path.isfile(path)
    else:
        is_path = os.path.isdir(path)

    print(f'{"OK:" if is_path else "ERROR:"} Path to {"file" if file else "folder"} {path} does{"" if is_path else " NOT"} exist')

    return is_path

In [3]:
HOMEPATH = getenv()

SRCPATH = os.path.join(HOMEPATH, 'Data', 'fMRIBreastData')

NIFTISRCFLDR = 'NiftiData'
CFGSRCFLDR = 'configFiles'

studypath = os.path.join(SRCPATH, NIFTISRCFLDR)
configpath = os.path.join(SRCPATH, CFGSRCFLDR)
# Check the path exist and are correct:
check_path_exist(studypath)
check_path_exist(configpath)

OK: Path to folder /Users/joseulloa/Data/fMRIBreastData/NiftiData does exist
OK: Path to folder /Users/joseulloa/Data/fMRIBreastData/configFiles does exist


True

In [4]:
DEBUGMODE = True 
BATCHMODE = True # TRUE: it runs the registration for all dataset within STUDYPATH; FALSE (DEFAULT): allows to pick a specific dataset to register

In [33]:
TEST_IDX = 9 # 1,2,3,4,5,6,7,8,9...
TEST_NRO = f'Test{TEST_IDX:03d}'

if TEST_IDX == 1:
    test_description = f'In test {TEST_NRO}, we assess the registration algorithm ELASTIX with parameters considered DEFAULT to 3D breas DCEMRI (i.e. Pars0032 from the model zoo). To replicate what is in the literature, we use the pre-contrast volume as the reference (i.e. fixed).'
    # Add here test's description and configuration files.
    registration_algorithm = 'Elastix' # ['Elastix', 'ANTs']
    # Which volume defines the reference (i.e. fixed) image space (1: pre-contrast, 2-6: post-contrast)
    fixed_volume_pos = 1 # an interesing test will be to use the 2nd phase as a reference (the one immediately after contrast injection)
    # The flag REGISTER_FIXED tells whether to register or not the fixed image with itself (in a future test I'm going to assess whether this at least equalises the signal intensity with the post-registered images)
    register_fixed = False # default value
    # Provide the configuration files that lies in the CONFIGPATH folder, they will then copied into the TEST_NRO folder
    # THEY MUST BE IN LOGICAL ORDER to load them in the parameters object!!! 
    config_files = ['Par0032_rigid.txt', 'Par0032_bsplines.txt']
    platform = 'any'
elif TEST_IDX == 2:
    test_description = f'In test {TEST_NRO}, we assess the effect of selecting the first post-contrast series as the reference (fixed) image. The fixed image is not affected and is copied straightaway from the raw Nifti data. Parameters are the considered DEFAULT for 3D Breast DCE (i.e. Pars0032 from the Model Zoo)'
    registration_algorithm = 'Elastix' # ['Elastix', 'ANTs']
    fixed_volume_pos = 2 # an interesing test is to use the 2nd phase as a reference (the one immediately after contrast injection)
    register_fixed = False # default value
    config_files = ['Par0032_rigid.txt', 'Par0032_bsplines.txt']
    platform = 'any'
elif TEST_IDX == 3:
    test_description = f'In test {TEST_NRO}, we assess the effect of registering the fixed image with itself. It is not expected to change the image contrast and even not to move any structure, but the test looks for assessing the image quality. When reviewing the previous tests, we notice a slight change in the image intensity, compared to the other series, most probably due the smoothing applied during registration. So the idea of the test is to see the same smoothing is applied to the fixed volume. The reference image is the pre-contrast series, same as TEST001. Parameters are the considered DEFAULT for 3D Breast DCE (i.e. Pars0032 from the Model Zoo)'
    registration_algorithm = 'Elastix'
    fixed_volume_pos = 1
    register_fixed = True
    config_files = ['Par0032_rigid.txt', 'Par0032_bsplines.txt']
    platform = 'any'
elif TEST_IDX == 4:
    test_description = f'In test {TEST_NRO}, we assess the same effect tested in TEST003, but using the 1st pre-contrast series as the reference volume. So apart from FIXED_VOLUME_POS, any other parameter is the same as TESTS003'
    registration_algorithm = 'Elastix'
    fixed_volume_pos = 2
    register_fixed = True
    config_files = ['Par0032_rigid.txt', 'Par0032_bsplines.txt']
    platform = 'any'
elif TEST_IDX == 5:
    test_description = f'In test {TEST_NRO}, we assess ANTs as a registration algorithm. We use the default parameters of the application, listed in the official documentation, and are saved in the config_files folder. The reference series is the pre-contrast volue and the fixed volume is NOT registerd to itself'
    registration_algorithm = 'ANTs'
    fixed_volume_pos = 1
    register_fixed = False
    config_files = ['DefaultANTspy.json']
    platform = 'any'
elif TEST_IDX == 6:
    test_description = f'In test {TEST_NRO}, we are not testing any registration parameter, but only setting up ANTs for batch processing. This test will be deleted once the code is ready.'
    registration_algorithm = 'ANTs'
    fixed_volume_pos = 1
    register_fixed = False
    config_files = ['DefaultANTspy.json']
    platform = 'any'
elif TEST_IDX == 7:
    test_description = f'In test {TEST_NRO}, we assess ANTs as a registration algorithm. We use the default parameters of the application, listed in the official documentation, and are saved in the config_files folder. The reference series is the 1st post-contrast volue and the fixed volume is NOT registerd to itself.'
    registration_algorithm = 'ANTs'
    fixed_volume_pos = 2
    register_fixed = False
    config_files = ['DefaultANTspy.json']
    platform = 'any'
elif TEST_IDX == 8:
    test_description = f'In test {TEST_NRO}, we assess ANTs as a registration algorithm. We use the default parameters of the application, listed in the official documentation, and are saved in the config_files folder. The reference series is the 1st post-contrast volue and the fixed volume is registerd to itself.'
    registration_algorithm = 'ANTs'
    fixed_volume_pos = 2
    register_fixed = True
    config_files = ['DefaultANTspy.json']
    platform = 'any'
elif TEST_IDX == 9:
    test_description = f'In test {TEST_NRO}, we perform the same tasks as in Testo 004, but things are run in Windows platform (Windows 10). We use the default parameters of the application, listed in the official documentation, and are saved in the config_files folder. The reference series is the 1st post-contrast volume and the fixed volume is registerd to itself.'
    registration_algorithm = 'Elastix'
    fixed_volume_pos = 2
    register_fixed = True
    config_files = ['Par0032_rigid.txt', 'Par0032_bsplines.txt']
    platform = 'win32'
elif TEST_IDX == 10:
    test_description = f'In test {TEST_NRO}, we evaluate the N4ITK correction algorithm (N4BiasFieldCorrectionImageFilter function in ITK). For now, we are testing it in a single dataset just as a PoC.'
    registration_algorithm = 'N4ITK'
    fixed_volume_pos = 0
    register_fixed = False
    config_files = ['']
    platform = 'any'
else:
    print(f'ERROR!!: {TEST_NRO} not yet defined')

print(test_description)

In test Test009, we perform the same tasks as in Testo 004, but things are run in Windows platform (Windows 10). We use the default parameters of the application, listed in the official documentation, and are saved in the config_files folder. The reference series is the 1st post-contrast volume and the fixed volume is registerd to itself.


In [34]:
outputfldr = os.path.join('tests',TEST_NRO)
savepath = os.path.join(SRCPATH, outputfldr)
# Check whether the folder SAVEPATH exists or not, if not, attempts to create it
os.makedirs(savepath, exist_ok=True)

dataset_to_process = {'testNro': TEST_NRO,
                      'study_path': studypath,
                      'save_path': savepath,
                      'data_path': os.path.join(savepath, 'datasets'),
                      'parameters_folder': 'parameters',
                      'intended_platform': platform,
                      'run_platform': sys.platform,
                      'registration_details': {'algorithm': registration_algorithm,
                                               'configuration_files': config_files,
                                               'register_fixed': register_fixed},
                      'fixed_volume_position': fixed_volume_pos,
                      'datasets': {}
                     }
# Create the top level folder(s) inside SAVEPATH:
os.makedirs(os.path.join(dataset_to_process['save_path'],dataset_to_process['parameters_folder']), exist_ok=True)

In [35]:
# Create description.json file from the descriptive variables defined in the previous cell:
description = {
    'TestID': TEST_NRO,
    'Summary': test_description,
    'Intended Platform': platform,
    'Run Platform': sys.platform,
    'Registration Details': {'algorithm': registration_algorithm,
                             'configuration parameters': config_files,
                             'reference volume': fixed_volume_pos,
                             'register fixed': register_fixed}
}

# Save the description as a JSON file in the output directory:
with open(os.path.join(dataset_to_process['save_path'], 'description.json'), 'w') as fp:
    json.dump(description, fp)

In [8]:
# List the patient list:
patients = list_folder_content(studypath)
print('Patient data folders:')
display_folder_list(patients)
if not BATCHMODE:
    # Pick up an option:
    patients_indices = range(len(patients))
    patientIDX = None
    while patientIDX not in patients_indices:
        patientIDX = input(f'Pick up a valid index to select a patient {tuple(patients_indices)} or type "x" to quit: ')
        if patientIDX == 'x':
            print(f'ERROR!: option "{patientIDX}" is not valid')
            break
        else:
            patientIDX = int(patientIDX)
        patient = patients[patientIDX]
        data_patient = os.path.join(studypath, patient)
        if DEBUGMODE:
            print(f'Patient {patient} selected contains the follow datasets:')
            print(display_tree(os.path.join(studypath, patient), header=True, string_rep=True, show_hidden=False, max_depth=4))
        patients = [patient]
else:
    print(f'Processing the whole data folder {studypath} as a batch process \n***Please be patient!!***')

Patient data folders:
[0] - DC-ANON97378
[1] - GL-ANON99397
[2] - JB-ANON18218
[3] - CR-ANON68760
[4] - EilB-ANON98269
[5] - RICE00-RICE001
[6] - NE-ANON89073
Processing the whole data folder /Users/joseulloa/Data/fMRIBreastData/NiftiData as a batch process 
***Please be patient!!***


It is expected the patient folder contains only 2 sub-folders: 
* PatientName-Pre-Treatment-\<visit-date\>
* PatientName-Post-Treatment-\<visit-date\>

and inside each of these sub-folders, there is the sequence number and the timepoints, where the corresponding Nifti file lives:
```
    PatientName-<Pre/Post>-Treatment-<visit-date>/
    ├── SeqNro/
        ├── 1/
            ├── <SeqNro>_<sequence-name>.nii.gz
        ├── 2/
            ├── <SeqNro>_<sequence-name>.nii.gz
        ├── 3/
            ├── <SeqNro>_<sequence-name>.nii.gz
        ├── 4/
            ├── <SeqNro>_<sequence-name>.nii.gz
        ├── 5/
            ├── <SeqNro>_<sequence-name>.nii.gz
        ├── 6/
            ├── <SeqNro>_<sequence-name>.nii.gz
```
Note that for each timepoint, there is no difference in the Nifti file name, it is only differentiated by the folder enclosing it

In [9]:
# Organise the code to run for a single dataset and then, embed it into a loop for batch processing...
# dataset_to_process = {}
for patient in patients:
    # loop over patient datafolder (pre- and post-treatment)
    data_patient = os.path.join(studypath, patient)
    dataset_to_process['datasets'][patient]={'output_path': patient, # data_patient.replace(studypath, savepath),
                                 'visits': {}}
    patient_visits = list_folder_content(data_patient)
    checks = True
    if DEBUGMODE:
        print(f'Subfolders inside {data_patient}:\n\t{patient_visits}')
    print(''.join(['*']*50))
    for patient_visit in patient_visits:
        visit_name = patient_visit.split('-')
        seq_path = os.path.join(data_patient, patient_visit)
        seq_nro = list_folder_content(seq_path)[0] 
        dataset_to_process['datasets'][patient]['visits'][''.join(visit_name[1:3])] = {'path': os.path.join(patient_visit, seq_nro),
                                                                           'path2fixed': ''}
        # Check the number of subfolders and depth are the expected ones:
        print(f'Checking {seq_path} contains only 1 folder...')
        check_nsequences_per_visit = check_time_points(seq_path, nmax=1, verbose=DEBUGMODE)
        if check_nsequences_per_visit is not None:
            print(f'Checking {seq_nro} contains the expected number of timepoints ...')
            check_timepoints_per_seq = check_time_points(os.path.join(seq_path, seq_nro), verbose=DEBUGMODE)
        else:
            checks = False

        if check_timepoints_per_seq is not None:
            dce_tpoints_path = list_folder_content(os.path.join(seq_path, seq_nro))
            dataset_to_process['datasets'][patient]['visits'][''.join(visit_name[1:3])]['path2moving'] = [None]*len(dce_tpoints_path)
            for time_point_i in dce_tpoints_path:
                print(f'Checking there is only one NIFTI file for each timepoint in {seq_nro}...')
                check_nfiles_per_timepoint = check_time_points(os.path.join(seq_path, seq_nro, time_point_i), nmax=1)
                get_nii_file = glob.glob(os.path.join(seq_path, seq_nro, time_point_i,'*.nii.gz'))
                if (check_nfiles_per_timepoint is not None) and (len(get_nii_file)==1):
                    print('Ready to load data...')
                    if int(time_point_i) == fixed_volume_pos:
                        dataset_to_process['datasets'][patient]['visits'][''.join(visit_name[1:3])]['path2fixed'] = get_nii_file[0]
                    dataset_to_process['datasets'][patient]['visits'][''.join(visit_name[1:3])]['path2moving'][int(time_point_i)-1] = get_nii_file[0]
                else:
                    checks = False
        else:
            checks = False

        if not checks:
            print(f'***ERROR***: There is something wrong with the data in {seq_path}, please check!!')
        print(''.join(['*']*100))
    if checks:
        print(f'All done loading NIFTI files from dataset {data_patient}')
    else:
        print(f'***ERROR***: There is something wrong with some (or all) data in {data_patient}, please check!!')
    print(''.join(['*']*100))
        
if DEBUGMODE:
    print(f'Details of the datasets to process:')
    print(json.dumps(dataset_to_process, indent=1))

Subfolders inside /Users/joseulloa/Data/fMRIBreastData/NiftiData/DC-ANON97378:
	['DC-Pre-Treatment-20230621', 'DC-Post-Treatment-20230726']
**************************************************
Checking /Users/joseulloa/Data/fMRIBreastData/NiftiData/DC-ANON97378/DC-Pre-Treatment-20230621 contains only 1 folder...
Folder /Users/joseulloa/Data/fMRIBreastData/NiftiData/DC-ANON97378/DC-Pre-Treatment-20230621 seems Ok
Listing folder content:
DC-Pre-Treatment-20230621/
└── 301/
Checking 301 contains the expected number of timepoints ...
Folder /Users/joseulloa/Data/fMRIBreastData/NiftiData/DC-ANON97378/DC-Pre-Treatment-20230621/301 seems Ok
Listing folder content:
301/
├── 1/
├── 2/
├── 3/
├── 4/
├── 5/
└── 6/
Checking there is only one NIFTI file for each timepoint in 301...
Folder /Users/joseulloa/Data/fMRIBreastData/NiftiData/DC-ANON97378/DC-Pre-Treatment-20230621/301/6 seems Ok
Ready to load data...
Checking there is only one NIFTI file for each timepoint in 301...
Folder /Users/joseulloa/D

In [10]:
# Registration parameters (this is the meat of the work!)
if dataset_to_process['registration_details']['algorithm'].lower() == 'ants':
    # ANTs
    # For details about possible values and description of parameters, see the help page: https://antspy.readthedocs.io/en/latest/registration.html
    # Default values (as listed in the hep page)
    dataset_to_process['par_set'] = {'type_of_transform': 'SyN', 
                                     'initial_transform': None, 
                                     'outprefix': '', 
                                     'mask': None,
                                     'moving_mask': None,
                                     'mask_all_stages': False,
                                     'grad_step': 0.2,
                                     'flow_sigma': 3, 
                                     'total_sigma': 0, 
                                     'aff_metric': 'mattes', 
                                     'aff_sampling': 32, 
                                     'aff_random_sampling_rate': 0.2, 
                                     'syn_metric': 'mattes', 
                                     'syn_sampling': 32, 
                                     'reg_iterations': (40, 20, 0),
                                     'aff_iterations': (2100, 1200, 1200, 10), 
                                     'aff_shrink_factors': (6, 4, 2, 1), 
                                     'aff_smoothing_sigmas': (3, 2, 1, 0), 
                                     'write_composite_transform': False, 
                                     'random_seed': None
                                    }

    # To ensure reproducibility of the results, set the random_seed to a constant value:
    dataset_to_process['par_set']['random_seed'] = 42 #(just to keep along with the pop-culture reference, e.g. https://medium.com/geekculture/the-story-behind-random-seed-42-in-machine-learning-b838c4ac290a
    # Save the parameters as a JSON file in the parameters folder:
    with open(os.path.join(dataset_to_process['save_path'], dataset_to_process['parameters_folder'], config_files[0]), 'w') as fp:
        json.dump(dataset_to_process['par_set'], fp)
elif dataset_to_process['registration_details']['algorithm'].lower() == 'elastix':
    # Elastix
    if DEBUGMODE:
        print(f'Define the parameters for the registration. Please wait...')
    dataset_to_process['par_set'] = itk.ParameterObject.New()
    for par_files in dataset_to_process['registration_details']['configuration_files']:
        dataset_to_process['par_set'].AddParameterFile(os.path.join(configpath, par_files))
        # Copy the parameters files to the output folder:
        shutil.copy2(os.path.join(configpath, par_files), os.path.join(dataset_to_process['save_path'], 
                                                                       dataset_to_process['parameters_folder'],
                                                                       par_files))
    
    if DEBUGMODE:
        print('Parameters for ELASTIX:')
        print(dataset_to_process['par_set'])
    # parameter_object.AddParameterMap(default_rigid_parameter_map)
else:
    print(f"Registration algorithm {dataset_to_process['registration_details']['algorithm']} not yet implemented. Please try again with a different option")

 ```
 Loop over the patientes to run the registration --> list(dataset_to_process['datasets'].keys())
 The workflow inside the loop is as follows:
  Loop over the visits  --> list(dataset_to_process['datasets'][list(dataset_to_process['datasets'].keys())[i]]['visits'].keys())
   Loads the Nifti file asigned to the fixed volume 
     Initialises the 4D volume
     Loops over the moving dataset --> dataset_to_process['datasets'][list(dataset_to_process['datasets'].keys())[i]]['visits'][list(dataset_to_process['datasets'][list(dataset_to_process['datasets'].keys())[j]]['visits'].keys())[z]]['path2moving']
        Loads the Nifti file
        if moving_index == fixed_index:
            skip registration and assigns the output to aux 
        else:
            runs registration
        saves the output data 
        concatenate to 4D volume
```     

In [11]:
print(f'Time at start: {time.ctime()}')
init_time = time.perf_counter()
for patient in dataset_to_process['datasets'].keys():
    start_registering_patient = time.perf_counter()
    for patient_visit in dataset_to_process['datasets'][patient]['visits']:
        start_registering_visit = time.perf_counter()
        patient_visit_outputpath = os.path.join(dataset_to_process['data_path'], patient, dataset_to_process['datasets'][patient]['visits'][patient_visit]['path'])
        print(patient_visit_outputpath)
        path2fixed = dataset_to_process['datasets'][patient]['visits'][patient_visit]['path2fixed']
        print(f'Fixed Volume: {path2fixed}')
        print(''.join(['-']*100))
        fixed_volume = itk.imread(path2fixed)
        if dataset_to_process['registration_details']['algorithm'].lower() == 'ants':
            # Convert ITK image to ANTs:
            fixed_volume_ants = itk2ants(fixed_volume)

        # Get the list of moving datasets:
        moving_datasets = dataset_to_process['datasets'][patient]['visits'][patient_visit]['path2moving']
        
        # ITK concatenation output: Defines the 4D volume from this fixed image
        # To stack the volumes, use the function TileFilter, following the example at 
        # https://examples.itk.org/src/filtering/imagegrid/create3dvolume/documentation 
        # However, as I found out the hard way, the method SetInput works in order, even if it is within a loop, it works lexicographically (i.e. ordinal numbers)
        # so we'll recycle the list required for ANTs and after the registration, populates the tile in another (much quicker) loop
        input_dimension = fixed_volume.GetImageDimension()
        pixel_type = itk.template(fixed_volume)[1][0]
        output_dimension = input_dimension + 1
        input_image_type = itk.Image[pixel_type, input_dimension]
        output_image_type = itk.Image[pixel_type, output_dimension]
        layout = [1, 1, 1, len(moving_datasets)]
        registered_tiles = itk.TileImageFilter[input_image_type, output_image_type].New()
        registered_tiles.SetLayout(layout)
        

        for idx, moving_set in enumerate(moving_datasets):
            # Create the sub-folder in the output directories:
            os.makedirs(os.path.join(patient_visit_outputpath, str(idx+1)), exist_ok=True)
            # If the index is the Fixed Volume, just copy it to the output folder:
            if ((idx+1) == fixed_volume_pos) and (not register_fixed):
                print(f'No registration needed, {moving_set} is the fixed volume')
                shutil.copy2(path2fixed, path2fixed.replace(dataset_to_process['study_path'], dataset_to_process['data_path']))
                registered_tiles.SetInput(idx, fixed_volume)
            else:
                # Any other case, just run the registration algorithm
                print(f'Registering dataset {moving_set} to reference volume {path2fixed}. Please wait...')
                start_registration_run = time.perf_counter()
                moving_volume = itk.imread(moving_set)
                if dataset_to_process['registration_details']['algorithm'].lower() == 'elastix':
                    warped_moving, result_transform_pars = itk.elastix_registration_method(fixed_volume , 
                                                                                           moving_volume, 
                                                                                           parameter_object=dataset_to_process['par_set'],
                                                                                           log_to_console=False) 
                elif dataset_to_process['registration_details']['algorithm'].lower() == 'ants':
                    moving_volume_ants = itk2ants(moving_volume)
                    registeredOutput = ants.registration(fixed=fixed_volume_ants , moving=moving_volume_ants, **dataset_to_process['par_set'])
                    warped_moving = ants2itk(registeredOutput['warpedmovout'])
                    
                    
                end_registration_run = time.perf_counter()
                elp_registration_single = end_registration_run - start_registration_run
                print(f'Elapsed time to register single volume (incl. loading the data): {elp_registration_single:0.2f}[s] ({timedelta(seconds=elp_registration_single)})')
                print(f'Adding registered volume to the 4D tile...')
                registered_tiles.SetInput(idx, warped_moving)
                print(f"Saving the output result in {moving_set.replace(dataset_to_process['study_path'], dataset_to_process['data_path'])}")
                itk.imwrite(warped_moving, moving_set.replace(dataset_to_process['study_path'], dataset_to_process['data_path']))
                print(f'Finished registering timepoint {idx+1}')
            print(''.join(['=']*100))
        end_registering_visit = time.perf_counter()
        elp_registration_visit = end_registering_visit - start_registering_visit
        print(f'Elapsed time to register all timepoints in a visit (incl. loading the data): {elp_registration_visit:0.2f}[s] ({timedelta(seconds=elp_registration_visit)})')
        
        if BATCHMODE:
            # When running in BatchMode adds a small pause to avoid the "IOStream.flush timed out" error
            print('Just breathing a little...')
            time.sleep(5)
            print('Ready to continue!')
        
        # Saving the 4D time series :
        start_saving_4Dvol = time.perf_counter()
        reg_writer = itk.ImageFileWriter[output_image_type].New()
        reg_writer.SetFileName(os.path.join(os.path.split(patient_visit_outputpath)[0], 
                                            '.'.join([os.path.split(dataset_to_process['datasets'][patient]['visits'][patient_visit]['path'])[0],
                                                      'nii.gz'])))
        reg_writer.SetInput(registered_tiles.GetOutput())
        reg_writer.Update()
        
        end_saving_4Dvol = time.perf_counter()
        elp_saving_4Dvol = end_saving_4Dvol - start_saving_4Dvol
        print(f'Elapsed time to save the 4D volume: {elp_saving_4Dvol:0.2f}[s] ({timedelta(seconds=elp_saving_4Dvol)})')

        print(''.join(['*']*100))
    end_registering_patient = time.perf_counter()
    elp_registration_patient = end_registering_patient - start_registering_patient
    print(f'Elapsed time to register a whole patient dataset: {elp_registration_patient:0.2f}[s] ({timedelta(seconds=elp_registration_patient)})')
    print(''.join(['§']*100))

final_time = time.perf_counter()
elp_whole_loop = final_time - init_time
print(f'Elapsed time to register the complete test {TEST_NRO} data folder: {elp_whole_loop:0.2f}[s] ({timedelta(seconds=elp_whole_loop)})')
print(f'Time at the end: {time.ctime()}')

Time at start: Thu Mar 21 09:02:07 2024
/Users/joseulloa/Data/fMRIBreastData/tests/Test008/datasets/DC-ANON97378/DC-Pre-Treatment-20230621/301
Fixed Volume: /Users/joseulloa/Data/fMRIBreastData/NiftiData/DC-ANON97378/DC-Pre-Treatment-20230621/301/2/301_dyn_ethrive.nii.gz
----------------------------------------------------------------------------------------------------
Registering dataset /Users/joseulloa/Data/fMRIBreastData/NiftiData/DC-ANON97378/DC-Pre-Treatment-20230621/301/1/301_dyn_ethrive.nii.gz to reference volume /Users/joseulloa/Data/fMRIBreastData/NiftiData/DC-ANON97378/DC-Pre-Treatment-20230621/301/2/301_dyn_ethrive.nii.gz. Please wait...
Elapsed time to register single volume (incl. loading the data): 97.87[s] (0:01:37.867555)
Adding registered volume to the 4D tile...
Saving the output result in /Users/joseulloa/Data/fMRIBreastData/tests/Test008/datasets/DC-ANON97378/DC-Pre-Treatment-20230621/301/1/301_dyn_ethrive.nii.gz
Finished registering timepoint 1
Registering datase